In [2]:
########################################
#
#             PATH SETUP
#
########################################

import sys
import os
sys.path.insert(0, os.path.abspath("."))

In [3]:
from lemer.rngs import MultiStreamRNG
from lemer.rvms import *

In [4]:
s1 = 0
s2 = 0
s3 = 0
multi_stream_rng = MultiStreamRNG(1234)
c = 10000
m = 1.5
for i in range(c):
    x = multi_stream_rng.random()
    s1 += cdfExponential(m=m, x=x)
    s2 += pdfExponential(m=m, x=x)
    s3 += idfExponential(m=m, u=x)

print(f"mean 1= {s1 / c}, mean 2= {s2 / c}, mean 3= {s3/c}")

mean 1= 0.26995601334253494, mean 2= 0.48669599110497475, mean 3= 1.502576617212265


In [5]:
print("hello i am under the water plz help me")

hello i am under the water plz help me
